In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

In [2]:
%run "./scripts/start.py"
df_full, df_train, df_test = IO_full()
print(len(df_train))
df_full, df_test = None, None # This notebook is for validation, no test set needed
df_train = make_simple_features(df_train)
simple_feats = ["bathrooms", "bedrooms", "price",        
                "num_photos", "num_features", "num_description_words",                    
                "created_month", "created_day", "created_hour"
            ]

49352


In [ ]:
import os
os.getcwd()

In [3]:
#read cluster_features
cluster_feat = pd.read_csv("./features/mk/train_kmean_ft.csv")
assert len(cluster_feat) == 49352

In [4]:
#read simple price features
price_f_path = "./features/ah/simple_price_features"
with open(price_f_path, "rb") as f:
    price_features = pickle.load(f)

In [5]:
#read simple price features
feature_path = "./features/ah/bow"
with open(feature_path, "rb") as f:
    bow_features = pickle.load(f)

In [6]:
# features do not have the same index as the original data set - need to reset index
df_train.reset_index(inplace = True)
cluster_feat.reset_index(inplace = True)

In [7]:
df_to_concat = [df_train, cluster_feat,price_features,bow_features]
df_train = pd.concat(df_to_concat, axis=1)
df_train= df_train.drop('index', 1)

## It was faster to just use your function that defines folds, than to set up a function to read files and assemble them in the right order. Results are the same and should be consistent, because random state is defined.

In [8]:
def make_folds(df_train):
    from sklearn.model_selection import StratifiedKFold
    col_feats = [col for col in df_train if col != "interest_level"]
    X = df_train[col_feats]
    target_num_map={"high":0, "medium":1, "low":2}
    y = df_train["interest_level"].apply(lambda x: target_num_map[x])
    skf = StratifiedKFold(n_splits=5, random_state = 0)
    skf.get_n_splits(X, y)
    return skf

In [9]:
def cv_results(df_train,features,classifier,folds):
    #the scoring here is defined as negative log-loss metric, don't know why, but easy to change - just multipy by -1
    cross_val_scores = cross_val_score(classifier, df_train[features], df_train["interest_level"], cv=folds, scoring='neg_log_loss')
    #print(-cross_val_scores)
    #print("average error", np.average(-cross_val_scores))
    return -cross_val_scores

In [ ]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [11]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

target_num_map={"high":0, "medium":1, "low":2}
df_train["interest_level_num"]=df_train["interest_level"].apply(lambda x: target_num_map[x]).values

folds = make_folds(df_train)

clf=RFC(n_estimators=100, random_state=42,n_jobs =-1)

features_benchmark = simple_feats + ["cluster_35"]
features_extend = simple_feats + ["cluster_35"] + list(bow_features.columns)
features_benchmark_pr = simple_feats + ["cluster_35"] + ["price_cut_offset","price_log_cut_offset"]
# need to do this, because there is a word "feature" which coincides with the feature column name in the original dataset
# this does really srew things up if not ammended!
features_extend = [x for x in features_extend if x != 'features']


cross_val_bench = cv_results(df_train,features_benchmark, clf, folds)
cross_val_extra_features = cv_results(df_train,features_extend, clf,folds)
cross_val_bench_price = cv_results(df_train,features_benchmark_pr, clf,folds)

print(np.average(cross_val_bench))
print(np.average(cross_val_extra_features))
print(np.average(cross_val_bench_price))

0.672027262563


NameError: name 'cross_val_scores_m_ft' is not defined